In [1]:
import os
import time
import logging
import yaml
import ast
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

from pprgo import utils
from pprgo import ppr
from pprgo import pprgo



In [2]:
# Set up logging
logger = logging.getLogger()
logger.handlers = []
ch = logging.StreamHandler()
formatter = logging.Formatter(
        fmt='%(asctime)s (%(levelname)s): %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S')
ch.setFormatter(formatter)
logger.addHandler(ch)
logger.setLevel('INFO')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
# !wget --show-progress -O data/reddit.npz https://ndownloader.figshare.com/files/23742119  #Reddit
# !wget --show-progress -O data/magc.npz https://figshare.com/ndownloader/files/24045741 #MAG-COARSE

# Download dataset

# Load config

In [4]:
with open('config_demo.yaml', 'r') as c:
    config = yaml.safe_load(c)

In [5]:
# For strings that yaml doesn't parse (e.g. None)
for key, val in config.items():
    if type(val) is str:
        try:
            config[key] = ast.literal_eval(val)
        except (ValueError, SyntaxError):
            pass

In [6]:
data_file           = config['data_file']           # Path to the .npz data file
data_file = 'data/cora_full.npz'
# data_file = 'data/pubmed.npz'
# data_file = 'data/reddit.npz'
# data_file = 'data/magc.npz'

split_seed          = config['split_seed']          # Seed for splitting the dataset into train/val/test
ntrain_div_classes  = config['ntrain_div_classes']  # Number of training nodes divided by number of classes
attr_normalization  = config['attr_normalization']  # Attribute normalization. Not used in the paper

alpha               = config['alpha']               # PPR teleport probability
alpha = 0.25
eps                 = config['eps']                 # Stopping threshold for ACL's ApproximatePR
topk                = config['topk']                # Number of PPR neighbors for each node
topk=32
ppr_normalization   = config['ppr_normalization']   # Adjacency matrix normalization for weighting neighbors

hidden_size         = config['hidden_size']         # Size of the MLP's hidden layer
nlayers             = config['nlayers']             # Number of MLP layers
weight_decay        = config['weight_decay']        # Weight decay used for training the MLP
dropout             = config['dropout']             # Dropout used for training

lr                  = config['lr']                  # Learning rate
max_epochs          = config['max_epochs']          # Maximum number of epochs (exact number if no early stopping)
batch_size          = config['batch_size']          # Batch size for training
batch_mult_val      = config['batch_mult_val']      # Multiplier for validation batch size

eval_step           = config['eval_step']           # Accuracy is evaluated after every this number of steps
run_val             = config['run_val']             # Evaluate accuracy on validation set during training

early_stop          = config['early_stop']          # Use early stopping
patience            = config['patience']            # Patience for early stopping

nprop_inference     = config['nprop_inference']     # Number of propagation steps during inference
inf_fraction        = config['inf_fraction']        # Fraction of nodes for which local predictions are computed during inference

# Load the data

In [7]:
start = time.time()
(adj_matrix, attr_matrix, labels,
 train_idx, val_idx, test_idx) = utils.get_data(
        f"{data_file}",
        seed=split_seed,
        ntrain_div_classes=ntrain_div_classes,
        normalize_attr=attr_normalization
)
try:
    d = attr_matrix.n_columns
except AttributeError:
    d = attr_matrix.shape[1]
nc = labels.max() + 1
print('Number of classes: ', nc)

# print('shape attibute matrix: ', attr_matrix.n_rows)
# print('Training: ', len(train_idx))
# print('Validation: ', len(val_idx))
# print('Testing: ', len(test_idx))
# print('train_idx: ', train_idx)

time_loading = time.time() - start
print(f"Runtime: {time_loading:.2f}s")

Number of classes:  70
Runtime: 0.22s


# Preprocessing: Calculate PPR scores

In [8]:

# core_numbers = None
core_numbers = np.load('coredata/cora-cores.npy')

In [9]:
idx_all_nodes = np.arange(adj_matrix.shape[0])

In [18]:
# compute the ppr vectors for train/val nodes using ACL's ApproximatePR

start = time.time()

# ppr_topk_train, core_topk_train, coreRank, mean_kn = ppr.topk_ppr_matrix(adj_matrix, alpha, eps, idx_all_nodes, None, core_numbers,
#                                  normalization=ppr_normalization)

# ppr_topk_train_top32, core_topk_train_top32, coreRank, mean_kn = ppr.topk_ppr_matrix(adj_matrix, alpha, eps, idx_all_nodes, topk, core_numbers,
#                                  normalization=ppr_normalization)


ppr_topk_train, core_topk_train, mean_kn = ppr.topk_ppr_matrix(adj_matrix, alpha, eps, train_idx, topk, core_numbers,
                                 normalization=ppr_normalization, elbow=True)

if run_val:
    topk_val = ppr.topk_ppr_matrix(adj_matrix, alpha, eps, val_idx, topk,
                                   normalization=ppr_normalization)
else:
    topk_val = None

time_preprocessing = time.time() - start
print(f"Runtime: {time_preprocessing:.2f}s")


Mean kn:  15.391428571428571
Runtime: 0.09s


In [11]:
import scipy.sparse

# scipy.sparse.save_npz('full_ppr.npz', ppr_topk_train)
# scipy.sparse.save_npz('full_core.npz', core_topk_train)
# scipy.sparse.save_npz('top32_ppr.npz', ppr_topk_train_top32)
# scipy.sparse.save_npz('top32_core.npz', core_topk_train_top32)

In [12]:
# ppr_topk_train = scipy.sparse.load_npz('top32_ppr.npz')
# core_topk_train = scipy.sparse.load_npz('top32_core.npz')

In [19]:
print(ppr_topk_train.shape)
print(core_topk_train.shape)
print(ppr_topk_train[0].nonzero()[0].shape)
print(core_topk_train[0].nonzero()[0].shape)


(1400, 18800)
(1400, 18800)
(52,)
(52,)


# Training: Set up model and train

In [20]:
start = time.time()
tf.reset_default_graph()
tf.set_random_seed(0)


gamma = 0.0

model = pprgo.PPRGo(d, nc, hidden_size, nlayers, lr, gamma, weight_decay, dropout, #adj_matrix, train_idx, TEMP, K, topk_train, intermediate_layer=100,
                    sparse_features=type(attr_matrix) is not np.ndarray)


In [21]:
sess = tf.compat.v1.Session()
with sess.as_default():
    tf.compat.v1.global_variables_initializer().run()
    nepochs, loss_hist, acc_hist, f1_hist = pprgo.train(
            sess=sess, model=model, attr_matrix=attr_matrix,
            train_idx=train_idx, val_idx=val_idx,
            # topk_train=ppr_topk_train[train_idx], topk_val=topk_val, core_topk_train=core_topk_train[train_idx],
            topk_train=ppr_topk_train, topk_val=topk_val, core_topk_train=core_topk_train,
            labels=labels, 
            max_epochs=max_epochs, batch_size=batch_size, batch_mult_val=batch_mult_val,
            eval_step=eval_step, early_stop=early_stop, patience=patience)
time_training = time.time() - start
logging.info('Training done.')
print(f"Runtime: {time_training:.2f}s")

2022-08-11 11:48:00 (INFO): Epoch 6, step 20: train 3.93249
2022-08-11 11:48:00 (INFO): Epoch 13, step 40: train 3.27489
2022-08-11 11:48:01 (INFO): Epoch 19, step 60: train 2.88875
2022-08-11 11:48:01 (INFO): Epoch 26, step 80: train 2.57202
2022-08-11 11:48:02 (INFO): Epoch 33, step 100: train 1.78412
2022-08-11 11:48:02 (INFO): Epoch 39, step 120: train 1.52578
2022-08-11 11:48:03 (INFO): Epoch 46, step 140: train 1.44289
2022-08-11 11:48:03 (INFO): Epoch 53, step 160: train 1.07313
2022-08-11 11:48:04 (INFO): Epoch 59, step 180: train 0.95348
2022-08-11 11:48:04 (INFO): Epoch 66, step 200: train 1.01308
2022-08-11 11:48:05 (INFO): Epoch 73, step 220: train 0.82139
2022-08-11 11:48:05 (INFO): Epoch 79, step 240: train 0.76500
2022-08-11 11:48:06 (INFO): Epoch 86, step 260: train 0.82824
2022-08-11 11:48:07 (INFO): Epoch 93, step 280: train 0.69431
2022-08-11 11:48:07 (INFO): Epoch 99, step 300: train 0.67494
2022-08-11 11:48:07 (INFO): Epoch 106, step 320: train 0.71245
2022-08-11 1

Runtime: 21.08s


In [22]:
# ppr_topk_train = scipy.sparse.load_npz('full_ppr.npz')
# core_topk_train = scipy.sparse.load_npz('full_core.npz')

print(ppr_topk_train.shape)
# print(core_topk_train.shape)
# print(ppr_topk_train[0].nonzero()[0].shape)
# print(core_topk_train[0].nonzero()[0].shape)

(1400, 18800)


# Inference (val and test)

In [23]:
start = time.time()
predictions, time_logits, time_propagation, logits = model.predict(
        # sess=sess, adj_matrix=adj_matrix, attr_matrix=attr_matrix, alpha=alpha, ppr_topk_test=ppr_topk_train[test_idx], core_topk_test=core_topk_train[test_idx],
        sess=sess, adj_matrix=adj_matrix, attr_matrix=attr_matrix, alpha=alpha, ppr_topk_test=None, core_topk_test=None,
        nprop=nprop_inference, inf_fraction=inf_fraction,
        ppr_normalization=ppr_normalization)
time_inference = time.time() - start
print(f"Runtime: {time_inference:.2f}s")

Inference gamma:  0.10288522
Runtime: 0.17s


In [24]:
print(len(predictions))
print(len(test_idx))

18800
3400


In [ ]:
deg = adj_matrix.sum(1).A1
deg_sqrt_inv = 1. / np.sqrt(np.maximum(deg, 1e-12))

print('logits: ', logits.shape)
print('adj_matrix: ', adj_matrix.shape)

# print(adj_matrix)
# print(deg_sqrt_inv)

# adj_matrix_norm =  adj_matrix.multiply(deg_sqrt_inv[:, None])

# print('x: ', x.shape, x )

# print('x: ', x.shape)

# x = np.arange(16).reshape(4,4)
# print(x)
# x = scipy.sparse.csr_matrix(x)

# # print(x)
# print(x.shape)

# y = np.arange(4)
# print(y)
# print(y[:, None].shape)


# w = x.multiply(y[:, None])
# print('w: ', w.shape, w.todense())

# z = y[:, None] * x
# print('z: ', z.shape)


# print(logits)

# # core_matrix = adj_matrix.multiply(coreRank)
# # normalized_core_matrix = core_matrix.multiply(1/core_matrix.sum(axis=1).A1[:, None])
# # print(normalized_core_matrix.sum(axis=1))

# M = adj_matrix.multiply(deg_sqrt_inv[:, None])
# A_inner = scipy.sparse.eye(adj_matrix.shape[0]) - (1 - alpha) * M
# result = scipy.sparse.linalg.inv(A_inner)

# print('result: ', result.shape, type(result))



# right_term = scipy.sparse.eye(adj_matrix.shape[0])
# adj_power = adj_matrix.multiply(deg_sqrt_inv[:, None])


# for k in range(1, nprop_inference):
#     if k ==1:
#         right_term += (1-alpha)  * adj_power
#     else:
#         adj_power = adj_power @ adj_power
#         right_term += np.power(1-alpha, k) * adj_power

# right_term = alpha * right_term

# adj_power = adj_power @ adj_power

# left_term = np.power(1-alpha, nprop_inference) * adj_power

# new_logits = (left_term +right_term) @ (deg_sqrt_inv[:, None] * logits)







In [ ]:
# print(deg_sqrt_inv[:, None].shape)
# print((adj_matrix).shape)

# deg_sqrt_inv[:, None] * 

In [ ]:
# xd =  deg_sqrt_inv[:, None] *(normalized_core_matrix * deg_sqrt_inv[:, None])
# print(xd.shape)
# adj = np.array([[0,1,1],[1,1,0], [1,0,1]])
# coreRank = np.array([3,7,9])

# print('adj: ', adj)
# print('coreRank: ', coreRank)

# coreMatrix = np.multiply(adj, coreRank)

# print('coreMatrix: ', coreMatrix)

# # print('sum: ', np.sum(coreMatrix, axis=1))

# z = np.multiply(coreMatrix, 1/ np.sum(coreMatrix, axis=1)[:,None])
# print(z)

# print(np.sum(z, axis=1))


In [ ]:
print(acc_hist)


# Collect and print results

In [25]:

acc_train = 100 * accuracy_score(labels[train_idx], predictions[train_idx])
acc_val = 100 * accuracy_score(labels[val_idx], predictions[val_idx])
acc_test = 100 * accuracy_score(labels[test_idx], predictions[test_idx])

# acc_test = 100 * accuracy_score(labels[test_idx], predictions)
# acc_train = 0.00
# acc_val = 0.00

f1_train = f1_score(labels[train_idx], predictions[train_idx], average='macro')
f1_val = f1_score(labels[val_idx], predictions[val_idx], average='macro')
f1_test = f1_score(labels[test_idx], predictions[test_idx], average='macro')
# f1_train = 0.00
# f1_val = 0.00
# f1_test = f1_score(labels[test_idx], predictions, average='macro')



#gpu_max_bytes = tf.contrib.memory_stats.MaxBytesInUse()
#gpu_memory = sess.run(gpu_max_bytes)
memory = utils.get_max_memory_bytes()

time_total = time_preprocessing + time_training + time_inference

In [26]:
print(f'''
Accuracy: Train: {acc_train:.1f}%, val: {acc_val:.1f}%, test: {acc_test:.1f}%
F1 score: Train: {f1_train:.3f}, val: {f1_val:.3f}, test: {f1_test:.3f}

Runtime: Preprocessing: {time_preprocessing:.2f}s, training: {time_training:.2f}s, inference: {time_inference:.2f}s -> total: {time_total:.2f}s
Memory: Main: {(memory/1024) / 2**30:.2f}GB
''')

#Memory: Main: {memory / 2**30:.2f}GB, GPU: {gpu_memory / 2**30:.3f}GB


Accuracy: Train: 99.3%, val: 61.2%, test: 60.5%
F1 score: Train: 0.992, val: 0.507, test: 0.495

Runtime: Preprocessing: 0.09s, training: 21.08s, inference: 0.17s -> total: 21.34s
Memory: Main: 0.53GB

